In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch


In [ ]:
df = pd.read_csv('/content/train (Autosaved).csv',encoding='latin-1')


In [ ]:
label_counts = df['target'].value_counts()

positive_count = label_counts.get(1, 0)
negative_count = label_counts.get(0, 0)

print(f"Positive samples: {positive_count}")
print(f"Negative samples: {negative_count}")

Positive samples: 10500
Negative samples: 12498


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        7598 non-null   float64
 1   keyword   7538 non-null   object 
 2   location  5070 non-null   object 
 3   text      22998 non-null  object 
 4   target    22998 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 898.5+ KB


In [ ]:
#Converting Data to Dataframe and defining X and labels
data_ = pd.DataFrame(df)
x = data_.drop('target', axis=1)
labels = df['target']

In [ ]:
x['text'] = x['text'].str.replace(r'http\S+|www\S+|@\S+', '', regex=True)
x['hashtag'] = x['text'].str.findall(r'#(\w+)').apply(lambda hashtags: ', '.join(hashtags) if hashtags else np.nan)
x['text'] = x['text'].str.replace('#', '', regex=False)
x['text_'] = x['text'].str.replace("[.,!]", "", regex=True)
x['text_'] = x['text_'].str.lower()
x['hashtag'] = x['hashtag'].str.lower()
x=x.drop(['id','text','location'],axis=1)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]  # Use str.isalpha()
    return " ".join(words)


In [ ]:
x['text_'] = x['text_'].apply(preprocess_text)


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
import re

def extract_keyword_using_embeddings(text):
    # Ensure the text is valid
    if not text or not isinstance(text, str) or not text.strip():
        return None  # or return an appropriate fallback value

    # Clean the text to retain only words
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.strip()  # Remove leading/trailing whitespace

    # Get the embeddings for the entire tweet
    tweet_embedding = model.encode([text])[0]

    # Split the tweet into words and get their embeddings
    words = text.split()
    if not words:  # Handle cases where splitting results in no valid words
        return None  # or return a fallback value

    word_embeddings = model.encode(words)

    # Calculate the cosine similarity between the tweet embedding and each word embedding
    similarities = cosine_similarity([tweet_embedding], word_embeddings)

    # Find the index of the word with the highest similarity
    best_word_index = np.argmax(similarities)

    # Return the word that has the highest similarity to the tweet
    return words[best_word_index]


In [ ]:
#2 - Replacing all  rows
x['keyword'] = x['keyword'].replace('', pd.NA)

# Replace all values in the 'keyword' column with embedding-generated keywords
x['keyword'] = x['text_'].apply(lambda text: extract_keyword_using_embeddings(text))

In [ ]:
pip install datasets

In [ ]:
from datasets import Dataset
x['labels']=labels
# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data_)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(x, test_size=0.2,shuffle=True, random_state=42)


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
def tokenize_function(example):
    return tokenizer(example["text_"], padding="max_length", truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/18398 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/4600 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format("torch")
val_dataset.set_format("torch")


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2,hidden_dropout_prob=0.3,  # Dropout for fully connected layers
attention_probs_dropout_prob=0.3  # Dropout for attention layers
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
 #Freeze the first 6 layers of RoBERTa (BERTweet)
for i, layer in enumerate(model.roberta.encoder.layer):
    if i < 6:  # Adjust number of layers to freeze as needed
        for param in layer.parameters():
            param.requires_grad = False


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Compute evaluation metrics for the model's predictions.
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # For classification models with logits
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')  # For binary classification
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./results",           # Output directory for model checkpoints
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save the model after each epoch
    learning_rate=1e-5,
    max_grad_norm=1.0,
    per_device_train_batch_size=24,  # Training batch size
    per_device_eval_batch_size=32,   # Validation batch size
    num_train_epochs=12,             # Total number of epochs
    weight_decay=0.03,               # Weight decay for regularization
    save_total_limit=1,              # Save only the last checkpoint
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,
    warmup_steps=500,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True,
    lr_scheduler_type="linear"
)

# Add EarlyStoppingCallback to the Trainer
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Number of epochs to wait for improvement in validation loss
)

# Initialize the Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[early_stopping],  # Add early stopping here
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-77-61e2c01a2801>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = "'/content/drive/My Drive/ndt_data"



Mounted at /content/drive


In [ ]:
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Validation Results:", results)



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.414300,0.370382,0.860217,0.791297,0.940163,0.859331
2,0.294900,0.308720,0.886304,0.842220,0.922451,0.880512
3,0.298700,0.316380,0.892391,0.846522,0.932025,0.887218
4,0.302100,0.316114,0.893261,0.843213,0.939684,0.888839
5,0.238600,0.296883,0.900000,0.858337,0.933940,0.894544
6,0.268300,0.280660,0.910652,0.894638,0.910483,0.902491
7,0.243400,0.289479,0.903913,0.863255,0.936812,0.898531
8,0.260400,0.308399,0.897174,0.846781,0.944471,0.892962
9,0.208300,0.302137,0.903261,0.862434,0.936333,0.897866


Validation Results: {'eval_loss': 0.280660480260849, 'eval_accuracy': 0.9106521739130434, 'eval_precision': 0.8946378174976481, 'eval_recall': 0.9104834849210148, 'eval_f1': 0.902491103202847, 'eval_runtime': 6.4811, 'eval_samples_per_second': 709.757, 'eval_steps_per_second': 22.218, 'epoch': 9.0}


In [ ]:
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define paths
drive_path = "/content/drive/My Drive/results"
os.makedirs(drive_path, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

print(f"Model and tokenizer saved to {drive_path}")



Model and tokenizer saved to /content/drive/My Drive/results


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import re
import matplotlib.pyplot as plt

# Function to detect past events
def is_past_event(text):
    past_time_patterns = [
        r'\b\d+\s+years? ago\b',
        r'\b\d+\s+months? ago\b',
        r'last\s+\w+',  # e.g., last week, last year
        r'\bin\s+\d{4}\b',  # mentions a specific year
        r'\bformer\b',  # Indicates past events
    ]
    for pattern in past_time_patterns:
        if re.search(pattern, text, flags=re.IGNORECASE):
            return True
    return False

# Function to adjust predictions
def adjust_predictions(text, prediction):
    if is_past_event(text) and prediction == "Positive":
        return "Negative"
    return prediction

# Load dataset
text_data = pd.read_csv('final_test.csv')

# Ensure the dataset has the necessary columns
assert 'tweet' in text_data.columns, "Column 'tweet' not found in dataset"
assert 'label' in text_data.columns, "Column 'label' not found in dataset"

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize inputs
inputs = tokenizer(
    list(text_data['tweet']),
    padding=True,
    truncation=True,
    return_tensors="pt"
).to(device)

# Get predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits, dim=1)

# Map class indices to labels
class_labels = ["Negative", "Positive"]  # Update as per your model's label mapping
predicted_labels = [class_labels[pred] for pred in predictions.cpu().numpy()]

# Adjust predictions
adjusted_predictions = [
    adjust_predictions(tweet, pred) for tweet, pred in zip(text_data['tweet'], predicted_labels)
]

# Convert true labels to string format for comparison
true_labels = text_data['label'].map({0: "Negative", 1: "Positive"}).tolist()

# Calculate metrics
precision = precision_score(true_labels, adjusted_predictions, pos_label="Positive", average='binary')
recall = recall_score(true_labels, adjusted_predictions, pos_label="Positive", average='binary')
accuracy = accuracy_score(true_labels, adjusted_predictions)
f1 = f1_score(true_labels, adjusted_predictions, pos_label="Positive", average='binary')

# Display results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot the metrics as bar charts
metrics = [precision, recall, accuracy, f1]
metric_names = ['Precision', 'Recall', 'Accuracy', 'F1 Score']

misclassifications = []
for tweet, true_label, pred_label in zip(text_data['tweet'], true_labels, adjusted_predictions):
    if true_label != pred_label:
        misclassifications.append((tweet, true_label, pred_label))

# Display misclassifications
if misclassifications:
    print("\nMisclassified Tweets:")
    for i, (tweet, true_label, pred_label) in enumerate(misclassifications):
        print(f"\n{i+1}. Tweet: {tweet}")
        print(f"True Label: {true_label}, Predicted Label: {pred_label}")
else:
    print("\nNo misclassifications found.")

Precision: 0.9317
Recall: 0.9983
Accuracy: 0.9637
F1 Score: 0.9639

Misclassified Tweets:

1. Tweet: Heavy winds and rain expected to last throughout the day. Secure loose outdoor items! #StormAlert #WindWarning
True Label: Positive, Predicted Label: Negative

2. Tweet: Lost my phone in the parking lot today. Luckily, someone turned it in! #PhoneLost #Thankful
True Label: Negative, Predicted Label: Positive

3. Tweet: Dropped my favorite mug this morning. RIP to the best coffee mug ever. #AccidentProne #MugDisaster
True Label: Negative, Predicted Label: Positive

4. Tweet: Slight delay getting to work because of some traffic. Hopefully, this won’t be a habit! #TrafficTrouble #MorningCommute
True Label: Negative, Predicted Label: Positive

5. Tweet: Locked myself out of the house today. The locksmith was very kind though! #HouseTroubles #ForgotTheKeys
True Label: Negative, Predicted Label: Positive

6. Tweet: Lost my favorite jacket in the restaurant. I hope someone finds it! #LostAndFo